In [1]:
import numpy as np
import math
import pandas as pd
import time
from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
root_dir = './'
doc_dir = './Document/'
query_dir = './Query/'

In [3]:
doc_names = []
query_names = []

with open('doc_list.txt', 'r') as f:
    doc_names = [line.strip() for line in f]

with open('query_list.txt', 'r') as f:
    query_names = [line.strip() for line in f]

In [4]:
training_set = {}

for i in range(len(doc_names)):
    with open(doc_dir + doc_names[i] , 'r') as f:
        next(f)
        next(f)
        next(f)
        words = []
        for line in f:
            words+=line.rstrip()[:-3].split(' ')
            
        training_set[i]=words

In [5]:
query_set = {}
for i in range(len(query_names)):
    with open(query_dir + query_names[i] , 'r') as f:
        words = []
        for line in f:
             words+=line.rstrip()[:-3].split(' ')
        query_set[i] = words

In [6]:
tf_dj_wi = {}
tf_qj_wi = {}
idf_wi = {}
vc_dj = {}
vc_qj = {}
word_list = {}

In [7]:
#tf_d
for j in range(len(training_set)):
    tf_dj_wi[j] = {}
    vc_dj[j] = vc = pd.value_counts(training_set[j])
    len_set = len(training_set[j])
    vc_index = list(vc.index)
    for i in range(len(vc_index)):
        tf_dj_wi[j][vc_index[i]] = vc[i] / len_set

In [8]:
#tf_q
for j in range(len(query_set)):
    tf_qj_wi[j] = {}
    vc_qj[j] = vc = pd.value_counts(query_set[j])
    len_set = len(query_set[j])
    vc_index = list(vc.index)
    for i in range(len(vc_index)):
        tf_qj_wi[j][vc_index[i]] = vc[i] / len_set

In [9]:
#word_list
all_index = []
for j in range(len(vc_dj)):
    all_index += list(vc_dj[j].index)
    
w = np.unique(all_index)
for i in range(len(w)):
    word_list[str(w[i])] = i

In [10]:
#idf
for j in range(len(training_set)):
    vc = vc_dj[j]
    vc_index = list(vc.index)
    
    for i in range(len(vc)):
        if vc_index[i] not in idf_wi:
            idf_wi[vc_index[i]] = 1
        else:
            idf_wi[vc_index[i]] = idf_wi[vc_index[i]] + 1

len_docs = len(training_set)
for word in idf_wi:
    idf_wi[word] = math.log10(len_docs/idf_wi[word])

In [11]:
def tf2vector(tf):
    vector = [0]*len(word_list)
    for word in tf:
        if word in word_list: #some word might not in word list of documents
            #vector = tf*idf 
            vector[word_list[word]] = tf[word]*idf_wi[word] 
    return vector

In [12]:
#tfidf of docs
tfidf_dj = [[0]*len(word_list)]*len(training_set)
for j in range(len(training_set)):
    tfidf_dj[j] = tf2vector(tf_dj_wi[j])

In [13]:
def sim(vector_q):
    return vector_q, cosine_similarity(vector_q, tfidf_dj)

In [46]:
it = 1
alpha = 0.3
beta = 0.7
gamma = 0.0
top = 5
last = 1

In [45]:
def rel_non_rel(arr):
    rel = arr[:top]
    non_rel = arr[len(arr)-last:]
    return rel, non_rel

In [18]:
##debug

arrs = [[1,0,0,0],[0,1,0,0],[0,0,0,1]]
arrs2 = [[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,0,1]]
cosine_similarity(arrs,arrs2)

# sum_rel = [0]*4
# for k in range(len(arrs)):
#     sum_rel = np.add(sum_rel, arrs[k])
    
# sum_rel =  arrs[0] +np.multiply(sum_rel, 2) - np.multiply([0, 0,1,1],0.2)
# print(sum_rel)



array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [16]:
def newQuery(origin_q, sim_arr):
    
    sim_arr = [x for _,x in reversed(sorted(zip(sim_arr,tfidf_dj)))]
    #split rel, non_rel
    rel, non_rel = rel_non_rel(sim_arr)
    
    #sum rel docs
    sum_rel = [0]*len(word_list)
    for k in range(len(rel)):
        sum_rel = np.add(sum_rel, rel[k])

    sum_non_rel = [0]*len(word_list)
    for k in range(len(non_rel)):
        sum_non_rel = np.add(sum_non_rel, non_rel[k])
    
    # np.multiply(arr[0], alpha) - np.multiply(non_rel,gamma)
    new_q = np.multiply(origin_q, alpha) + np.multiply(sum_rel, beta/len(rel)) - np.multiply(sum_non_rel,gamma/len(non_rel))


    return new_q
    

In [17]:
tfidf_q  = [tf2vector(tf_qj_wi[j]) for j in range(len(tf_qj_wi))]

In [18]:
ori_all, s_all = sim(tfidf_q)

In [29]:
new_querys = [newQuery(ori_all[i],s_all[i]) for i in range(len(s_all))]

800

In [33]:
ori_all, s_all  = sim(new_querys)

In [34]:
result = s_all

In [47]:
for k in range(0,it):
    #initial
    if k == 0:
        ori_all, s_all = sim(tfidf_q)
    new_querys = [newQuery(ori_all[i],s_all[i]) for i in range(len(s_all))]
    ori_all, s_all  = sim(new_querys)

result = s_all

In [48]:
with open(time.ctime()+'submission.txt', 'w') as t:
    t.write('Query,RetrievedDocuments\n')
    for j in range(len(query_names)):
        ans = ' '.join([x for _,x in reversed(sorted(zip(result[j],doc_names)))])
        t.write('%s,%s\n' %(query_names[j],ans))